In [490]:
import re
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.datasets import imdb

from keras.utils.np_utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [491]:
data=pd.read_csv(r"C:\Users\JAI\Downloads\sentiment labelled sentences (1)\sentiment labelled sentences\imdb_labelled.csv")
data.columns=["text","Sentiment"]
data.head(10)

,text,Sentiment
0,Not sure who was more lost - the flat characte...,0.0
1,Attempting artiness with black & white and cle...,0.0
2,Very little music or anything to speak of.,0.0
3,The best scene in the movie was when Gerardo i...,1.0
4,"The rest of the movie lacks art, charm, meanin...",0.0
5,Wasted two hours.,0.0
6,Saw the movie today and thought it was a good ...,1.0
7,A bit predictable.,0.0
8,Loved the casting of Jimmy Buffet as the scien...,1.0
9,And those baby owls were adorable.,1.0


In [492]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
data['text'] = data['text'].apply(lambda x: x.lower())


for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=1000, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X.shape

(999, 59)

In [493]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print('Shape of training samples:',X_train.shape,Y_train.shape)
print('Shape of testing samples:',X_test.shape,Y_test.shape)

Shape of training samples: (669, 59) (669, 2)
Shape of testing samples: (330, 59) (330, 2)


In [494]:
model = Sequential()
model.add(Embedding(max_fatures, 128 ,input_length = X.shape[1]))
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 59, 128)           256000    
_________________________________________________________________
lstm_9 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
Total params: 387,842
Trainable params: 387,842
Non-trainable params: 0
_________________________________________________________________
None


In [495]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
 - 61s - loss: 0.6905 - acc: 0.5590
Epoch 2/5
 - 5s - loss: 0.6398 - acc: 0.7399
Epoch 3/5
 - 4s - loss: 0.4929 - acc: 0.7982
Epoch 4/5
 - 3s - loss: 0.2816 - acc: 0.8909
Epoch 5/5
 - 3s - loss: 0.1653 - acc: 0.9357


In [499]:
text = 'Nah I dont think he goes to usf, he lives around here though'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['text']

tester['text'] = tester['text'].apply(lambda x: x.lower())

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')
    
test = np.array([test])

prediction = model.predict(test)

print('Prediction value:',prediction.any())
print('Prediction value:',prediction.all())
print('Prediction value:',prediction[0])

for i in prediction:
    print("spam")
#else:
 #   print('nope')

Prediction value: True
Prediction value: True
Prediction value: [0.00402446 0.99597555]
spam


In [497]:
 score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Accuracy: %.2f" % (acc))

Score: 0.73
Accuracy: 0.72


In [349]:
spamData=pd.read_csv(r"E:\workspace\spam.csv",encoding='latin-1')
spamData=spamData.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
spamData.head(10)
File=spamData['v2']
for i in range(0,5572):
    print(File[i])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&Cs apply 08452810075over18s
U dun say so early hor... U c already then say...
Nah I dont think he goes to usf, he lives around here though
FreeMsg Hey there darling its been 3 weeks now and no word back! Id like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv
Even my brother is not like to speak with me. They treat me like aids patent.
As per your request Melle Melle (Oru Minnaminunginte Nurungu Vettam) has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
Had your mobile 11 months or more? U R entitled to 

Hi. Customer Loyalty Offer:The NEW Nokia6650 Mobile from ONLY å£10 at TXTAUCTION! Txt word: START to No: 81151 & get yours Now! 4T&Ctxt TC 150p/MTmsg
Wish i were with you now!
Haha mayb ure rite... U know me well. Da feeling of being liked by someone is gd lor. U faster go find one then all gals in our group attached liao.
Yes i will be there. Glad you made it.
Do well :)all will for little time. Thing of good times ahead:
Just got up. have to be out of the room very soon. Û_. i hadnt put the clocks back til at 8 i shouted at everyone to get up and then realised it was 7. wahay. another hour in bed.
Ok. There may be a free gym about.
Men like shorter ladies. Gaze up into his eyes.
Dunno he jus say go lido. Same time 930.
I promise to take good care of you, princess. I have to run now. Please send pics when you get a chance. Ttyl!
U are subscribed to the best Mobile Content Service in the UK for å£3 per 10 days until you send STOP to 82324. Helpline 08706091795
Is there a reason weve n

Waiting 4 my tv show 2 start lor... U leh still busy doing ur report?
Oh ho. Is this the first time u use these type of words
Am I the only one who doesnt stalk profiles?
Ever green quote ever told by Jerry in cartoon \A Person Who Irritates u Always Is the one Who Loves u Vry Much But Fails to Express It...!..!! :-) :-) gud nyt"
Yes i thought so. Thanks.
But if she.s drinkin im ok.
Just wondering, the others just took off
Night has ended for another day, morning has come in a special way. May you smile like the sunny rays and leaves your worries at the blue blue bay. Gud mrng
What do you do, my dog ? Must I always wait till the end of your day to have word from you ? Did you run out of time on your cell already?
Happy new year to u too!
Hey...Great deal...Farm tour 9am to 5pm $95/pax, $50 deposit by 16 May
Eat jap done oso aft ur lect wat... ÌÏ got lect at 12 rite... 
Hey babe! I saw you came online for a second and then you disappeared, what happened ?
Da my birthdate in certificate 

7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st \Ur Lovely Friendship\"... good morning dear"
Loan for any purpose å£500 - å£75,000. Homeowners + Tenants welcome. Have you been previously refused? We can still help. Call Free 0800 1956669 or text back help
BIG BROTHER ALERT! The computer has selected u for 10k cash or #150 voucher. Call 09064018838. NTT PO Box CRO1327 18+ BT Landline Cost 150ppm mobiles vary
;-( oh well, c u later
My uncles in Atlanta. Wish you guys a great semester.
No dear i do have free messages without any recharge. Hi hi hi
Dont search love, let love find U. Thats why its called falling in love, bcoz U dont force yourself, U just fall and U know there is smeone to hold U... BSLVYL
I dun believe u. I thk u told him.
Do you know why god created gap between your fingers..? So that, One who is made for you comes &amp; fills those gaps by holding your hand with LOVE..!
Yes:)sura in sun tv.:)lol.
Arun can u tra

For ur chance to win a å£250 cash every wk TXT: ACTION to 80608. Ts&Cs www.movietrivia.tv custcare 08712405022, 1x150p/wk.
Well I might not come then...
Long after I quit. I get on only like 5 minutes a day as it is.
Then its most likely called Mittelschmertz. Google it. If you dont have paracetamol dont worry it will go.
Well at this right Im gonna have to get up and check todays steam sales/pee so text me when you want me to come get you
Just arrived, see you in a couple days &lt;3
K, wat s tht incident?
Yeah get the unlimited
cThen i thk shd b enuff.. Still got conclusion n contents pg n references.. Ill b doing da contents pg n cover pg..
Forgot it takes me 3 years to shower, sorry. Where you at/your phone dead yet?
ÌÏ got wat to buy tell us then Ì_ no need to come in again.
When you are big..| God will bring success.
UÛªve Bin Awarded å£50 to Play 4 Instant Cash. Call 08715203028 To Claim. EVERY 9th Player Wins Min å£50-å£500. OptOut 08718727870
ham"
Not from this campus. Are you

Once a fishrman woke early in d mrng. It was very dark. He waited a while &amp; found a sack ful of stones. He strtd throwin thm in2 d sea 2 pass time. Atlast he had jus 1stone, sun rose up &amp; he found out tht those r nt stones, those were diamonds. Moral:\Dont wake up early in d mrng GOOD night"
Claim a 200 shopping spree, just call 08717895698 now! Have you won! MobStoreQuiz10ppm
Then ur physics get a-?
Dear friends, sorry for the late information. Today is the birthday of our loving Ar.Praveesh. for more details log on to face book and see. Its his number + &lt;#&gt; . Dont miss a delicious treat.
How r Ì_ going to send it to me?
Can you do online transaction?
Dear got train and seat mine lower seat
Let me know if you need anything else. Salad or desert or something... How many beers shall i get?
Wat r u doing?
WHORE YOU ARE UNBELIEVABLE.
Want to funk up ur fone with a weekly new tone reply TONES2U 2 this text. www.ringtones.co.uk, the original n best. Tones 3GBP network operator

U R THE MOST BEAUTIFUL GIRL IVE EVER SEEN. U R MY BABY COME AND C ME IN THE COMMON ROOM
O we cant see if we can join denis and mina? Or does denis want alone time
Sen told that he is going to join his uncle finance in cbe
Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke 
Call me, i am senthil from hsbc.
Especially since i talk about boston all up in my personal statement, lol! I woulda changed that if i had realized it said nyc! It says boston now.
Indeed and by the way it was either or - not both !
Urgent -call 09066649731from Landline. Your complimentary 4* Ibiza Holiday or å£10,000 cash await collection SAE T&Cs PO BOX 434 SK3 8WP 150ppm 18+
Holy living christ what is taking you so long
ÌÏ thk of wat to eat tonight.
Thanx. Yup we coming back on sun. Finish dinner going back 2 hotel now. Time flies, were tog 4 exactly a mth today. Hope well haf many more mths to come...
Were on the opposite side from where we dropped you off
Yup. Izzit still raining hea

In [350]:
pred=[]
for i in range(0,5572):
    text = File[i]
    tester = np.array([text])
    tester = pd.DataFrame(tester)
    tester.columns = ['text']

    tester['text'] = tester['text'].apply(lambda x: x.lower())
    tester['text'] = tester['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
    max_fatures = 2000
    test = tokenizer.texts_to_sequences(tester['text'].values)
    test = pad_sequences(test)

    if X.shape[1]>test.shape[1]:
        test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')
    
    test = np.array([test])

    prediction = model.predict(test)
    print(prediction)
    print(prediction.argmax())
    pred.append(prediction.argmax())
    i=i+1

[[0.03228083 0.9677192 ]]
1
[[0.909203   0.09079702]]
0
[[0.8443148  0.15568522]]
0
[[0.05204873 0.9479513 ]]
1
[[0.02947067 0.9705293 ]]
1
[[0.00657641 0.9934236 ]]
1
[[0.97012615 0.02987384]]
0
[[0.89845276 0.10154722]]
0
[[0.98686814 0.01313182]]
0
[[0.93030393 0.06969608]]
0
[[0.16632085 0.83367914]]
1
[[0.4972369 0.5027631]]
1
[[0.9408351  0.05916487]]
0
[[0.3722461 0.6277539]]
1
[[0.59973925 0.40026072]]
0
[[0.98241514 0.01758491]]
0
[[0.9724782  0.02752177]]
0
[[0.950457   0.04954304]]
0
[[0.12784654 0.8721534 ]]
1
[[0.76433825 0.23566176]]
0
[[0.98144364 0.01855638]]
0
[[0.8967508  0.10324923]]
0
[[0.6787623 0.3212377]]
0
[[0.16028622 0.83971375]]
1
[[0.8574392  0.14256082]]
0
[[0.9490647  0.05093529]]
0
[[0.35151753 0.6484825 ]]
1
[[0.01388955 0.98611045]]
1
[[0.6261909 0.3738091]]
0
[[0.8228576  0.17714237]]
0
[[0.99035066 0.00964934]]
0
[[0.98057127 0.01942865]]
0
[[0.23487872 0.7651212 ]]
1
[[0.9675057  0.03249429]]
0
[[0.9381075  0.06189248]]
0
[[0.7199857  0.28001425]]
0


ValueError: Error when checking input: expected embedding_8_input to have 2 dimensions, but got array with shape (1, 1, 66)

In [364]:
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#spamData['v2'] = spamData['v2'].apply(lambda x: x.lower())

#for idx,row in data.iterrows():
 #   row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=1000, split=' ')
tokenizer.fit_on_texts(spamData['v2'].values)
Z = tokenizer.texts_to_sequences(spamData['v2'].values)
Z = pad_sequences(X)

In [365]:
S=model.predict(Z)

In [366]:
print(S)
S.shape

[[0.99652076 0.00347928]
 [0.99460536 0.00539465]
 [0.79912996 0.20087007]
 ...
 [0.92417747 0.07582258]
 [0.9935916  0.00640838]
 [0.98738086 0.01261916]]


(999, 2)